<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/objectivos_reactiva_vers%C3%A3o%20ciclo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install pyxlsb

In [3]:
#nome dos ficheiros
versao_ciclo="V1_Tratar_Cycle_Detail_RRR__PQ_MASTER_Upfront.xlsb"
resumo_nome=".Resumo templates_abril24.xlsb"
nome_folha_objectivo="Objetivos reativa"

In [4]:
#fazer o upload dos ficheiros
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()

Saving .Resumo templates_abril24.xlsb to .Resumo templates_abril24.xlsb


Saving V1_Tratar_Cycle_Detail_RRR__PQ_MASTER_Upfront.xlsb to V1_Tratar_Cycle_Detail_RRR__PQ_MASTER_Upfront.xlsb


In [5]:
#ler a dataframe da 1ª versão do ciclo

ciclo=pd.read_excel(versao_ciclo, sheet_name="Dados carregados").iloc[:, :7]

In [6]:
#formatar a dataframe da 1ª versão do ciclo
ciclo2=ciclo.drop(ciclo.iloc[:, 3].name, axis=1)

indice_max=ciclo2.isna().all(axis=1).idxmax()

ciclo3=ciclo2.iloc[:indice_max, :]

ciclo3.columns=["Tipo", "Objective name", "Objective value", "Tipo 2", "Objective name 2", "Objective value 2"]

ciclo3.head()


,Tipo,Objective name,Objective value,Tipo 2,Objective name 2,Objective value 2
0,Decisores,Exclusivos,1.5,Renegociação,Exclusivos,0.5
1,Decisores,MATRIZ BASE MTR,1.5,Renegociação,Convergente_MATRIZ BASE 2LO,0.0
2,Decisores,MATRIZ BASE C2C,1.5,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...,0.0
3,Decisores,MATRIZ BASE 2LO,1.5,Renegociação,Não Convergente_Com CM_3P Baixo Valor_MATRIZ B...,0.0
4,Decisores,MATRIZ BASE OUT,1.5,Renegociação,Não Convergente_DTH_MATRIZ BASE 2LO,0.0


In [7]:
#dividir dataframe no ciclo em 2 - ciclo_half1 inclui "decisores" e "retenção" - ciclo_half2 inclui "renegociação" - não são extraídas as colunas "Objective value"
ciclo_half1=ciclo3.iloc[:, :2]

ciclo_half2=ciclo3.iloc[:, 3:5]


In [8]:
#inspeccionar dataframe "ciclo_half1"
ciclo_half1

,Tipo,Objective name
0,Decisores,Exclusivos
1,Decisores,MATRIZ BASE MTR
2,Decisores,MATRIZ BASE C2C
3,Decisores,MATRIZ BASE 2LO
4,Decisores,MATRIZ BASE OUT
...,...,...
64,Retenção,Não Convergente_DTH_AT_TM OUT
65,Retenção,Não Convergente_Potenciais UpSellers_AT_TM OUT
66,Retenção,Convergente_AT_TM OUT
67,NaN,Convergente_EXCLUSIVOS


In [9]:
#inspeccionar dataframe "ciclo_half2"
ciclo_half2

,Tipo 2,Objective name 2
0,Renegociação,Exclusivos
1,Renegociação,Convergente_MATRIZ BASE 2LO
2,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...
3,Renegociação,Não Convergente_Com CM_3P Baixo Valor_MATRIZ B...
4,Renegociação,Não Convergente_DTH_MATRIZ BASE 2LO
...,...,...
64,NaN,NaN
65,NaN,NaN
66,NaN,NaN
67,NaN,NaN


In [10]:
#inspeccionar "ciclo_half2" - remover n/a na dataframe "ciclo_half2"
ciclo_half2=ciclo_half2.iloc[:ciclo_half2.isna().all(axis=1).idxmax(), :]

print(ciclo_half2.shape)
ciclo_half2.head()

(58, 2)


,Tipo 2,Objective name 2
0,Renegociação,Exclusivos
1,Renegociação,Convergente_MATRIZ BASE 2LO
2,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...
3,Renegociação,Não Convergente_Com CM_3P Baixo Valor_MATRIZ B...
4,Renegociação,Não Convergente_DTH_MATRIZ BASE 2LO


In [11]:
#substituir nan por vazio em "ciclo_half1" e "ciclo_half2"
def substituir_na(db):
  db2=db.fillna("")
  return db2

ciclo_half1=ciclo_half1.apply(substituir_na)
ciclo_half2=ciclo_half2.apply(substituir_na)

In [12]:
#função "chave" para criar 1 chave entre a coluna "tipo" e "objective name"
def chave(db, col1, col2):
  lista1=db[col1].tolist()
  lista2=db[col2].tolist()

  lista=list(map(lambda i, j: i + "_" + j if len(i) else j, lista1, lista2))

  db["chave"]=lista

  return db["chave"]

ciclo_half1["chave"]=chave(ciclo_half1, "Tipo", "Objective name")
ciclo_half2["chave"]=chave(ciclo_half2, "Tipo 2", "Objective name 2")


In [13]:
#ler ficheiro ".resumo templates"
resumo=pd.read_excel(resumo_nome, sheet_name=nome_folha_objectivo).iloc[:, :6]

resumo2=resumo.dropna()

print(resumo2.shape)
resumo2.head()

(122, 6)


,SFID,Type,Objective Name,Begin Date,End Date,Objective\n(Value)
0,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Qualquer_Qualquer...,45383,45412.0,0.5
1,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Convergente_Qualq...,45383,45412.0,0.0
2,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Não Convergente_C...,45383,45412.0,0.0
3,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Não Convergente_C...,45383,45412.0,0.0
4,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Não Convergente_Q...,45383,45412.0,0.0


In [14]:
#formatar ficheiro ".resumo templates" - substituir caracteres (ex: RRR_GCO_04.2024_)

caracter_subs=resumo2.iloc[:, 2].apply(lambda i: "_".join(i.split("_")[:3]) + "_").values[0]

caracter_subs
resumo2["Objective Name"]=resumo2["Objective Name"].replace(regex=caracter_subs + "*", value="")

resumo2.head()


<ipython-input-14-4a52c3956e64>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resumo2["Objective Name"]=resumo2["Objective Name"].replace(regex=caracter_subs + "*", value="")


,SFID,Type,Objective Name,Begin Date,End Date,Objective\n(Value)
0,RETENÇÃO RESIDENCIAL,Group,Renegociação_Qualquer_Qualquer_Qualquer_EXCLUS...,45383,45412.0,0.5
1,RETENÇÃO RESIDENCIAL,Group,Renegociação_Convergente_Qualquer_Qualquer_MAT...,45383,45412.0,0.0
2,RETENÇÃO RESIDENCIAL,Group,Renegociação_Não Convergente_Com CM_3P Alto Va...,45383,45412.0,0.0
3,RETENÇÃO RESIDENCIAL,Group,Renegociação_Não Convergente_Com CM_3P Baixo V...,45383,45412.0,0.0
4,RETENÇÃO RESIDENCIAL,Group,Renegociação_Não Convergente_Qualquer_DTH_MATR...,45383,45412.0,0.0


In [15]:
#verificar se existem n/a na dataframe "resumo2"
resumo2.isna().sum()

SFID                  0
Type                  0
Objective Name        0
Begin Date            0
End Date              0
Objective\n(Value)    0
dtype: int64

In [16]:
#substituir a substring "Qualquer_" na dataframe "resumo2" e criar nova coluna: "Objective Name 2"
resumo2["chave"]=resumo2["Objective Name"].str.replace("Qualquer_", "", regex=False)

<ipython-input-16-d1417397bbe5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resumo2["chave"]=resumo2["Objective Name"].str.replace("Qualquer_", "", regex=False)


In [17]:
#inspeccionar a dataframe "resumo2" após alterações
resumo2["chave"]

0                                Renegociação_EXCLUSIVOS
1               Renegociação_Convergente_MATRIZ BASE 2LO
2      Renegociação_Não Convergente_Com CM_3P Alto Va...
3      Renegociação_Não Convergente_Com CM_3P Baixo V...
4       Renegociação_Não Convergente_DTH_MATRIZ BASE 2LO
                             ...                        
117    Retenção_Não Convergente_Sem CM_3P Baixo Valor...
118               Retenção_Não Convergente_DTH_AT_TM 2LO
119    Retenção_Não Convergente_Potenciais UpSellers_...
120                       Retenção_Convergente_AT_TM 2LO
121                                  Decisores_AT_TM 2LO
Name: chave, Length: 122, dtype: object

In [18]:
#verificar quais as linhas na dataframe "resumo" que não estão na dataframe ciclo - "chave" -> ciclo e "Objective Name 2" -> resumo
def check_linhas(db1, db2, str, str2="chave"):
  lista_ausente= [i for i in db1.loc[db1[str2].str.contains(str, case=False), str2].tolist() if i not in db2.loc[db2[str2].str.contains(str, case=False), str2].tolist()]
  lista_presente=[i for i in db1.loc[db1[str2].str.contains(str, case=False), str2].tolist() if i in db2.loc[db2[str2].str.contains(str, case=False), str2].tolist()]
  return lista_ausente, lista_presente

check_linhas(ciclo_half1, resumo2, "retenção")

(['Retenção_Exclusivos'],
 ['Retenção_Não Convergente_Com CM_3P Alto Valor_MATRIZ BASE 2LO',
  'Retenção_Não Convergente_Sem CM_3P Alto Valor_MATRIZ BASE 2LO',
  'Retenção_Não Convergente_Com CM_3P Baixo Valor_MATRIZ BASE 2LO',
  'Retenção_Não Convergente_Sem CM_3P Baixo Valor_MATRIZ BASE 2LO',
  'Retenção_Não Convergente_DTH_MATRIZ BASE 2LO',
  'Retenção_Não Convergente_Potenciais UpSellers_MATRIZ BASE 2LO',
  'Retenção_Convergente_MATRIZ BASE 2LO',
  'Retenção_Não Convergente_Com CM_3P Alto Valor_MATRIZ BASE C2C',
  'Retenção_Não Convergente_Sem CM_3P Alto Valor_MATRIZ BASE C2C',
  'Retenção_Não Convergente_Com CM_3P Baixo Valor_MATRIZ BASE C2C',
  'Retenção_Não Convergente_Sem CM_3P Baixo Valor_MATRIZ BASE C2C',
  'Retenção_Não Convergente_DTH_MATRIZ BASE C2C',
  'Retenção_Não Convergente_Potenciais UpSellers_MATRIZ BASE C2C',
  'Retenção_Convergente_MATRIZ BASE C2C',
  'Retenção_Não Convergente_3P Alto Valor_MATRIZ BASE IN',
  'Retenção_Não Convergente_3P Baixo Valor_MATRIZ BASE IN

In [172]:
def pontuacao(db1, db2, lista, str2="chave", str3="Objective\n(Value)"):

  for str in lista:
    label, pontuacao=db2.loc[db2[str2].str.contains(str, case=False), str2], db2.loc[db2[str2].str.contains(str, case=False), str3]

    for i in range(len(label.tolist())):

      try:
        label2=label.iloc[i]
        indice=db1[db1[str2]==label2].index
        db1.loc[indice, "Objectivo valor"]=pontuacao.iloc[i]

      except KeyError:
        print(f"label 2 is {label2}")


    #actualização do "exclusivo"

  ind_excl_ciclo=db1[db1[str2].str.contains("exclusivo", case=False)].index #retornar o índice a preencher na versão ciclo

  if len(lista)==1:#procedimento renegociação
    str=lista[0]
    val_excl_res=db2.loc[(db2[str2].str.contains("exclusivo", case=False)) \
    & (db2[str2].str.contains(str, case=False)), str3].values

    db1.loc[ind_excl_ciclo, "Objectivo valor"]=[val_excl_res]*3

  else: #procedimento para decisor + retenção
    lista2=["decisor"] + ["retenção"]*3
    iterator=zip(ind_excl_ciclo, lista2)

    for ind, str in iterator:
      val_excl_res=db2.loc[(db2[str2].str.contains("exclusivo", case=False)) \
      & (db2[str2].str.contains(str, case=False)), str3].values

      db1.loc[ind, "Objectivo valor"]=val_excl_res


  #substituir eventuais valores N/A
  db1.fillna("")
  return db1

retencao_decisor=pontuacao(ciclo_half1, resumo2, lista=["retenção", "decisor"])
renegociacao=pontuacao(ciclo_half2, resumo2, lista=["renegociação"])

In [173]:
retencao_decisor.iloc[:,-1].sum()

34.330000000000005

In [174]:
renegociacao.iloc[:,-1].sum()

9.07

In [ ]:
#exportar dataframes "retencao_decisor" e "renegociacao"

In [171]:
retencao_decisor.drop(retencao_decisor.iloc[:, -1].name, axis=1, inplace=True)
renegociacao.drop(renegociacao.iloc[:, -1].name, axis=1, inplace=True)